### Testing similarity between individuals

In [4]:
import random
import ast

In [3]:
def create_individual(effects, effect_structure):
    n_effects_chosen = random.randint(1, len(effects) - 2)
    selected_effects = random.sample(effects, n_effects_chosen)
    
    individ = {}
    for effect in selected_effects:
        if effect in effect_structure:
            structure = effect_structure[effect]
            individ[effect] = {
                param: round(random.uniform(range_[0], range_[1]), 2) 
                for param, (_, range_) in structure.items()
            }
    return individ

In [2]:
n_effects = 6
effects = [i for i in range(n_effects)]
effect_structure = {
    0: { "rate_hz": ('float', (0.0, 100.0)), },# Chorus
    1: { "delay_seconds": ('float', (0.0, 10.0)), },# Delay
    2: { "drive_db": ('float', (0.0, 50.0)), },# Distortion
    3: { "gain_db": ('float', (-50.0, 50.0)) },# Gain
    4: { "depth": ('float', (0.0, 1.0)), },# Phaser
    5: { "wet_level": ('float', (0.0, 1.0)), },# Reverb
}
effects_map = {
    0: 'Chorus',
    1: 'Delay',
    2: 'Distortion',
    3: 'Gain',
    4: 'Phaser',
    5: 'Reverb',
}

In [4]:
individ1 = create_individual(effects, effect_structure)
individ2 = create_individual(effects, effect_structure)

print(individ1)
print(individ2)

# Extract common keys
common_keys = individ1.keys() & individ2.keys()

similarity_value = 0.0
for key in common_keys:
    weight = min(1 / len(individ1), 1 / len(individ2))
    #print(f'Key: {key}, weight: {weight}')
    
    inner_dict = effect_structure[key]
    lower_v = None
    max_v = None
    for param, value in inner_dict.items():
        lower_v = value[1][0]
        max_v = value[1][1]
                
    param_name = list(effect_structure[key].keys())[0]
    r = individ1[key][param_name]
    x = individ2[key][param_name]
    
    #print("r = ", r)
    #print("x = ", x)
    #print("low limit = ", lower_v)
    #print("max limit = ", max_v)
    
    distance_to_lower_v = abs(r - lower_v)
    distance_to_max_v = abs(r - max_v)
    
    limit_chosen = None
    if distance_to_lower_v < distance_to_max_v:
        limit_chosen = max_v
    else:
        limit_chosen = lower_v
    
    #print("abs(x - r) = ", abs(x - r))
    #print("abs(r - limit_chosen) = ", abs(r - limit_chosen))
    
    obtained_value_normalized = 1 - abs(x - r)/abs(r - limit_chosen)
    similarity_value += weight * obtained_value_normalized
    
print(similarity_value)

{4: {'depth': 0.68}, 5: {'wet_level': 0.7}, 1: {'delay_seconds': 7.55}, 0: {'rate_hz': 61.65}}
{3: {'gain_db': -22.02}, 2: {'drive_db': 37.43}}
0.0


In [5]:
def similarity_individuals(individ1, individ2):
    common_keys = individ1.keys() & individ2.keys()
    similarity = 0.0
    for key in common_keys:
        weight = min(1 / len(individ1), 1 / len(individ2))        
        inner_dict = effect_structure[key]
        param_name, (range_name, (lower_v, max_v)) = next(iter(inner_dict.items()))
         
        param_name = list(effect_structure[key].keys())[0]
        r = individ1[key][param_name]
        x = individ2[key][param_name]
        
        limit_chosen = max_v if abs(r - lower_v) < abs(r - max_v) else lower_v
        
        obtained_value_normalized = 1 - abs(x - r)/abs(r - limit_chosen)
        similarity += weight * obtained_value_normalized
        
    return similarity

In [6]:
individ1 = create_individual(effects, effect_structure)
individ2 = create_individual(effects, effect_structure)
similarity = similarity_individuals(individ1, individ2)

print(individ1)
print(individ2)
print(similarity)

{3: {'gain_db': 21.13}}
{5: {'wet_level': 0.61}, 1: {'delay_seconds': 9.72}, 4: {'depth': 0.64}}
0.0


### Testing similarity on Dataset

In [7]:
import pandas as pd

df = pd.read_csv('./results/testing_results_now.csv')
similarities = []
for index, row in df.iterrows():
    applied_effects = row['applied_effects']
    ga_effects = row['GA_effects']

    # Convert the string representations to dictionaries
    if pd.notnull(applied_effects) and pd.notnull(ga_effects):
        try:
            applied_effects = ast.literal_eval(applied_effects)
            ga_effects = ast.literal_eval(ga_effects)
        except (ValueError, SyntaxError) as e:
            print(f"Error converting row {index}: {e}")
            similarities.append(None)
            continue

        # Perform the similarity calculation
        similarities.append(similarity_individuals(applied_effects, ga_effects))
    else:
        # If either field is missing, append a None or other placeholder value
        similarities.append(None)
        
# Add the 'similarity_indv' column to the DataFrame
df['similarity_indv'] = pd.Series(similarities)

# Save the updated DataFrame back to a CSV file
df.to_csv('./results/testing_results_now.csv', index=False)

### Test individual similarity on random baseline

In [7]:
import pandas as pd

df = pd.read_csv('./results/dataset_audios_guitar_random_BIG.csv')
similarities = []
for index, row in df.iterrows():
    applied_effects = row['applied_effects']
    ga_effects = row['random_effects']

    # Convert the string representations to dictionaries
    if pd.notnull(applied_effects) and pd.notnull(ga_effects):
        try:
            applied_effects = ast.literal_eval(applied_effects)
            ga_effects = ast.literal_eval(ga_effects)
        except (ValueError, SyntaxError) as e:
            print(f"Error converting row {index}: {e}")
            similarities.append(None)
            continue

        # Perform the similarity calculation
        similarities.append(similarity_individuals(applied_effects, ga_effects))
    else:
        # If either field is missing, append a None or other placeholder value
        similarities.append(None)
        
# Add the 'similarity_indv' column to the DataFrame
df['similarity_indv'] = pd.Series(similarities)

# Save the updated DataFrame back to a CSV file
df.to_csv('./results/dataset_audios_guitar_random_BIG.csv', index=False)

In [ ]:
df = pd.read_csv('./results/dataset_audios_guitar_random.csv')
